vcf_file.vcf (https://raw.githubusercontent.com/samtools/htslib/develop/test/tabix/vcf_file.vcf)

In [1]:
from pathlib import Path
import shutil, gzip
import pandas as pd

In [2]:
ROOT = Path(r"C:\Users\Jisoo Chae\Self project\3 VCF Conversion Tool\project1")
DATA = ROOT / "data"
RESULTS = ROOT / "results"
EXPORTS = ROOT / "exports"
for p in (DATA, RESULTS, EXPORTS):
    p.mkdir(parents=True, exist_ok=True)

print("ROOT    :", ROOT)
print("DATA    :", DATA)
print("RESULTS :", RESULTS)
print("EXPORTS :", EXPORTS)


ROOT    : C:\Users\Jisoo Chae\Self project\3 VCF Conversion Tool\project1
DATA    : C:\Users\Jisoo Chae\Self project\3 VCF Conversion Tool\project1\data
RESULTS : C:\Users\Jisoo Chae\Self project\3 VCF Conversion Tool\project1\results
EXPORTS : C:\Users\Jisoo Chae\Self project\3 VCF Conversion Tool\project1\exports


In [3]:
# 1) Source VCF in ROOT (uncompressed .vcf)
src_vcf = ROOT / "vcf_file.vcf"
assert src_vcf.exists(), f"Input VCF not found at {src_vcf}. Put vcf_file.vcf in the ROOT folder."

# Copy into RESULTS so the notebook artifacts are self-contained
vcf_path = RESULTS / "public_variants.vcf"
shutil.copy(src_vcf, vcf_path)
print("Using VCF:", vcf_path)

Using VCF: C:\Users\Jisoo Chae\Self project\3 VCF Conversion Tool\project1\results\public_variants.vcf


In [4]:
# 2) Parse VCF (supports .vcf and .vcf.gz)
is_gz = vcf_path.suffix == ".gz"
open_fn = (lambda p: gzip.open(p, "rt")) if is_gz else (lambda p: open(p, "rt", encoding="utf-8", errors="ignore"))

def _parse_info(info_str: str):
    """Parse INFO field into a dict (best effort)."""
    d = {}
    for item in info_str.split(";"):
        if not item:
            continue
        if "=" in item:
            k, v = item.split("=", 1)
            d[k] = v
        else:
            d[item] = True
    return d

records = []
with open_fn(vcf_path) as fh:
    for line in fh:
        if not line or line.startswith("#"):  # skip header/meta
            continue
        parts = line.rstrip("\n").split("\t")
        if len(parts) < 8:
            continue  # skip malformed lines
        chrom, pos, vid, ref, alt, qual, flt, info = parts[:8]
        first_alt = alt.split(",")[0]
        info_dict = _parse_info(info)
        # convert DP/AF if present
        dp = pd.to_numeric(info_dict.get("DP"), errors="coerce")
        af = pd.to_numeric(info_dict.get("AF"), errors="coerce")
        records.append({
            "chrom": chrom,
            "pos": int(pos),
            "id": None if vid == "." else vid,
            "ref": ref,
            "alt": alt,
            "qual": None if qual == "." else float(qual),
            "filter": flt,
            "info": info,
            "type": "SNP" if len(ref) == 1 and len(first_alt) == 1 else "INDEL",
            "DP": dp,
            "AF": af,
        })

cols = ["chrom","pos","id","ref","alt","qual","filter","info","type","DP","AF"]
df = pd.DataFrame.from_records(records, columns=cols)
print("Parsed variants:", len(df))
display(df.head(10) if not df.empty else df)

Parsed variants: 15


,chrom,pos,id,ref,alt,qual,filter,info,type,DP,AF
0,1,3000150,None,C,T,59.2,PASS,AN=4;AC=2,SNP,NaN,NaN
1,1,3000151,None,C,T,59.2,PASS,AN=4;AC=2,SNP,NaN,NaN
2,1,3062915,id3D,GTTT,G,12.9,q10,"DP4=1,2,3,4;AN=4;AC=2;INDEL;STR=test",INDEL,NaN,NaN
3,1,3062915,idSNP,G,"T,C",12.6,test,"TEST=5;DP4=1,2,3,4;AN=3;AC=1,1",SNP,NaN,NaN
4,1,3106154,None,CAAA,C,342.0,PASS,AN=4;AC=2,INDEL,NaN,NaN
5,1,3106154,None,C,CT,59.2,PASS,AN=4;AC=2,INDEL,NaN,NaN
6,1,3157410,None,GA,G,90.6,q10,AN=4;AC=4,INDEL,NaN,NaN
7,1,3162006,None,GAA,G,60.2,PASS,AN=4;AC=2,INDEL,NaN,NaN
8,1,3177144,None,G,T,45.0,PASS,AN=4;AC=2,SNP,NaN,NaN
9,1,3177144,None,G,.,45.0,PASS,AN=4;AC=0,SNP,NaN,NaN


In [5]:
# 3) Save CSV & Excel to EXPORTS
csv_path  = EXPORTS / "public_variants.csv"
xlsx_path = EXPORTS / "public_variants.xlsx"
df.to_csv(csv_path, index=False)

# Pick Excel engine if available
engine = None
try:
    import openpyxl  # noqa
    engine = "openpyxl"
except Exception:
    try:
        import xlsxwriter  # noqa
        engine = "xlsxwriter"
    except Exception:
        engine = None

if engine is None:
    print("\n⚠️ Excel engine not found. CSV saved.\n"
          "Install one and re-run to get Excel:\n"
          "  conda run -n caris-mini conda install -y -c conda-forge openpyxl\n"
          "  (or) conda run -n caris-mini conda install -y -c conda-forge xlsxwriter")
else:
    with pd.ExcelWriter(xlsx_path, engine=engine) as xw:
        df.to_excel(xw, sheet_name="variants", index=False)

        if not df.empty:
            top = df.sort_values("qual", ascending=False, na_position="last").head(50)
            top.to_excel(xw, sheet_name="top_variants", index=False)

            chrom_counts = (
                df["chrom"]
                .value_counts(dropna=False)
                .rename_axis("chrom")
                .reset_index(name="n_variants")
            )
            chrom_counts.to_excel(xw, sheet_name="chrom_counts", index=False)

            summary = pd.DataFrame({
                "metric": ["n_variants", "n_snp", "n_indel", "qual_mean", "qual_median"],
                "value": [len(df),
                          int((df["type"] == "SNP").sum()),
                          int((df["type"] == "INDEL").sum()),
                          float(df["qual"].mean(skipna=True)) if "qual" in df else None,
                          float(df["qual"].median(skipna=True)) if "qual" in df else None]
            })
        else:
            summary = pd.DataFrame({
                "metric": ["n_variants", "n_snp", "n_indel", "qual_mean", "qual_median"],
                "value": [0, 0, 0, None, None]
            })

        summary.to_excel(xw, sheet_name="summary", index=False)

    print("Wrote CSV :", csv_path)
    print("Wrote XLSX:", xlsx_path)

Wrote CSV : C:\Users\Jisoo Chae\Self project\3 VCF Conversion Tool\project1\exports\public_variants.csv
Wrote XLSX: C:\Users\Jisoo Chae\Self project\3 VCF Conversion Tool\project1\exports\public_variants.xlsx
